# Complex Aggregation

### Objectives

+ Group on multiple columns
+ Perform a group by aggregation on a subset of the DataFrame columns
+ Perform a group by with multiple aggregation columns
+ Perform a group by with different aggregations for different columns
+ Group by multiple columns, aggregate multiple columns and apply multiple aggregation functions simultaneously

In [1]:
import pandas as pd
import numpy as np

## City of Houston Employee Data
We will import the City of Houston employee data, set the index to **`UNIQUE_ID`**, and add a years of experience column as both the number of years and as a category like we did in a previous notebook.

In [2]:
employee = pd.read_csv('data/employee.csv', index_col='UNIQUE_ID', parse_dates=['HIRE_DATE', 'JOB_DATE'])

employee['YEARS_EXPERIENCE'] = (pd.Timestamp('2016-12-1') - employee['HIRE_DATE']) / pd.Timedelta(1, 'Y')
employee['YEARS_EXPERIENCE'] = employee['YEARS_EXPERIENCE'].round(1)
employee['EXPERIENCE_LEVEL'] =  pd.cut(employee['YEARS_EXPERIENCE'], 
                                       bins=[0, 5, 15, 100], 
                                       labels=['Novice', 'Experienced', 'Senior'])
employee.head()

POSITION_TITLE                     DEPARTMENT  \
UNIQUE_ID                                                               
5906       ASSISTANT DIRECTOR (EX LVL)    Municipal Courts Department   
364                  LIBRARY ASSISTANT                        Library   
1286                    POLICE OFFICER  Houston Police Department-HPD   
8789                 ENGINEER/OPERATOR  Houston Fire Department (HFD)   
8542                       ELECTRICIAN    General Services Department   

           BASE_SALARY             RACE EMPLOYMENT_TYPE  GENDER  \
UNIQUE_ID                                                         
5906          121862.0  Hispanic/Latino       Full Time  Female   
364            26125.0  Hispanic/Latino       Full Time  Female   
1286           45279.0            White       Full Time    Male   
8789           63166.0            White       Full Time    Male   
8542           56347.0            White       Full Time    Male   

          EMPLOYMENT_STATUS  HIRE_DATE   JOB_DATE  YEARS_EXPERIENCE  \
UNIQUE_ID                                                             
5906                 Active 2006-06-12 2012-10-13              10.5   
364                  Active 2000-07-19 2010-09-18              16.4   
1286                 Active 2015-02-03 2015-02-03               1.8   
8789                 Active 1982-02-08 1991-05-25              34.8   
8542                 Active 1989-06-19 1994-10-22              27.5   

          EXPERIENCE_LEVEL  
UNIQUE_ID                   
5906           Experienced  
364                 Senior  
1286                Novice  
8789                Senior  
8542                Senior

### Terminology of a group by aggregation
The last notebook focused on grouping with a single column and aggregating on a single column with one aggregation function. Let's make sure we have our terminology straight. If we wanted to take the mean salary of both genders we would have the following:

Terminology:  
+ **Grouping column** - GENDER
+ **Aggregation column** - BASE_SALARY
+ **Aggregation method** - mean

All aggregations can be broken down into those three components. There are a few ways to write the syntax for this in pandas. One of which is the following.
> **`df.groupby(['columns', 'to', 'group'])['aggregating', 'columns'].agg(['aggregating, 'functions'])`**

### Simple groupby
Before we do any complex grouping, lets actually find the mean salary of both genders.

In [3]:
employee.groupby('GENDER')['BASE_SALARY'].mean()

GENDER
Female    52168.339688
Male      57354.611917
Name: BASE_SALARY, dtype: float64

In [4]:
employee.groupby('GENDER')['BASE_SALARY'].agg('mean')

GENDER
Female    52168.339688
Male      57354.611917
Name: BASE_SALARY, dtype: float64

## Grouping with Multiple Columns
We can pass as list to the groupby method to group by two or more columns. To find the mean salary for each RACE and GENDER, do the following.

In [5]:
employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].mean()

RACE                               GENDER
American Indian or Alaskan Native  Female    60238.800000
                                   Male      60305.400000
Asian/Pacific Islander             Female    63226.300000
                                   Male      61033.906667
Black or African American          Female    48915.421233
                                   Male      51082.074074
Hispanic/Latino                    Female    46503.316176
                                   Male      54782.819018
Others                             Female    63785.000000
                                   Male      38771.000000
White                              Female    66793.352941
                                   Male      63940.388119
Name: BASE_SALARY, dtype: float64

## New Dimensions of the Index
The examples above should look quite different than a normal Series or DataFrame. Each index is two dimensional. Technically, they are called **`levels`**. Each level has a name. Those names are **`RACE`** and **`GENDER`**. This new index object is a **`MultiIndex`**. There is much more to MultiIndexes and they are discussed in a separate notebook.

## Keeping the grouping columns as columns
pandas automatically puts all grouping columns into the index. To avoid this behavior, use the **`as_index`** parameter.

In [6]:
# use as_index = False to make a dataframe
employee.groupby(['RACE', 'GENDER'], as_index=False)['BASE_SALARY'].mean()

RACE  GENDER   BASE_SALARY
0   American Indian or Alaskan Native  Female  60238.800000
1   American Indian or Alaskan Native    Male  60305.400000
2              Asian/Pacific Islander  Female  63226.300000
3              Asian/Pacific Islander    Male  61033.906667
4           Black or African American  Female  48915.421233
5           Black or African American    Male  51082.074074
6                     Hispanic/Latino  Female  46503.316176
7                     Hispanic/Latino    Male  54782.819018
8                              Others  Female  63785.000000
9                              Others    Male  38771.000000
10                              White  Female  66793.352941
11                              White    Male  63940.388119

## Aggregating multiple columns
It is possible to perform an aggregation on multiple columns. To do this, simply pass multiple column names as strings to the indexing operator.

Let's find the mean salary and years of experience for each gender.

In [7]:
employee.groupby('GENDER')['BASE_SALARY', 'YEARS_EXPERIENCE'].mean()

BASE_SALARY  YEARS_EXPERIENCE
GENDER                                
Female  52168.339688         12.653400
Male    57354.611917         14.719757

In [12]:
employee.groupby('GENDER')['BASE_SALARY', 'YEARS_EXPERIENCE'].agg('mean')

BASE_SALARY  YEARS_EXPERIENCE
GENDER                                
Female  52168.339688         12.653400
Male    57354.611917         14.719757

## Using multiple aggregation functions
All our previous examples have only used a single aggregating function, like the mean or the sum. It is possible to use multiple aggregation functions by passing them too a list inside of the **`agg`** method. Let's find the mean, min and max of the salary by gender.

In [8]:
employee.groupby('GENDER')['BASE_SALARY'].agg(['mean', 'min', 'max'])

mean      min       max
GENDER                                 
Female  52168.339688  24960.0  178331.0
Male    57354.611917  26104.0  275000.0

# Use multiple columns for grouping and aggregating
We can use lists both inside the **`groupby`** method and outside to group and aggregate on. Let's group by race and gender and find the mean of both the salary and years of experience.

In [ ]:
employee.groupby(['RACE','GENDER'])['BASE_SALARY', 'YEARS_EXPERIENCE'].mean()

In [ ]:
employee.groupby(['RACE','GENDER'])['BASE_SALARY', 'YEARS_EXPERIENCE'].agg('mean')

## Using multiple columns and aggregating with multiple functions

In [9]:
employee.groupby(['RACE','GENDER'])['BASE_SALARY'].agg(['mean', 'min', 'max'])

mean      min       max
RACE                              GENDER                                 
American Indian or Alaskan Native Female  60238.800000  26125.0   98536.0
                                  Male    60305.400000  26125.0   81239.0
Asian/Pacific Islander            Female  63226.300000  26125.0  130416.0
                                  Male    61033.906667  27914.0  163228.0
Black or African American         Female  48915.421233  24960.0  150416.0
                                  Male    51082.074074  26125.0  275000.0
Hispanic/Latino                   Female  46503.316176  26125.0  126115.0
                                  Male    54782.819018  26104.0  165216.0
Others                            Female  63785.000000  63785.0   63785.0
                                  Male    38771.000000  38771.0   38771.0
White                             Female  66793.352941  27955.0  178331.0
                                  Male    63940.388119  26125.0  210588.0

## Aggregating multiple columns with multiple functions

In [10]:
employee.groupby(['RACE'])['BASE_SALARY', 'YEARS_EXPERIENCE'].agg(['mean', 'min', 'max'])

BASE_SALARY                     \
                                           mean      min       max   
RACE                                                                 
American Indian or Alaskan Native  60272.100000  26125.0   98536.0   
Asian/Pacific Islander             61660.304762  26125.0  163228.0   
Black or African American          50137.801493  24960.0  275000.0   
Hispanic/Latino                    52345.562771  26104.0  165216.0   
Others                             51278.000000  38771.0   63785.0   
White                              64419.799012  26125.0  210588.0   

                                  YEARS_EXPERIENCE             
                                              mean  min   max  
RACE                                                           
American Indian or Alaskan Native        13.027273  2.8  25.1  
Asian/Pacific Islander                   13.785981  0.5  39.4  
Black or African American                13.249000  0.5  48.0  
Hispanic/Latino                          12.308958  0.5  41.1  
Others                                    2.550000  1.1   4.0  
White                                    17.081353  0.6  57.9

## Grouping with multiple columns, aggregating multiple columns and using multiple aggregating functions

In [11]:
employee.groupby(['GENDER','RACE'])['BASE_SALARY', 'YEARS_EXPERIENCE'].agg(['mean', 'min', 'max'])

BASE_SALARY                     \
                                                  mean      min       max   
GENDER RACE                                                                 
Female American Indian or Alaskan Native  60238.800000  26125.0   98536.0   
       Asian/Pacific Islander             63226.300000  26125.0  130416.0   
       Black or African American          48915.421233  24960.0  150416.0   
       Hispanic/Latino                    46503.316176  26125.0  126115.0   
       Others                             63785.000000  63785.0   63785.0   
       White                              66793.352941  27955.0  178331.0   
Male   American Indian or Alaskan Native  60305.400000  26125.0   81239.0   
       Asian/Pacific Islander             61033.906667  27914.0  163228.0   
       Black or African American          51082.074074  26125.0  275000.0   
       Hispanic/Latino                    54782.819018  26104.0  165216.0   
       Others                             38771.000000  38771.0   38771.0   
       White                              63940.388119  26125.0  210588.0   

                                         YEARS_EXPERIENCE             
                                                     mean  min   max  
GENDER RACE                                                           
Female American Indian or Alaskan Native        12.583333  2.8  21.0  
       Asian/Pacific Islander                   14.470000  1.0  35.3  
       Black or African American                13.652131  0.5  37.6  
       Hispanic/Latino                          11.063309  0.6  37.3  
       Others                                    1.100000  1.1   1.1  
       White                                    13.018519  0.8  41.5  
Male   American Indian or Alaskan Native        13.560000  3.7  25.1  
       Asian/Pacific Islander                   13.519481  0.5  39.4  
       Black or African American                12.937722  0.5  48.0  
       Hispanic/Latino                          12.816716  0.5  41.1  
       Others                                    4.000000  4.0   4.0  
       White                                    17.869120  0.6  57.9

## Aggregating different columns with different functions
It is possible to use different aggregating functions for the different columns you are performing the aggregation on. You simply pass a dictionary of column names mapped to a list of functions you would like to aggregate on.

In [13]:
employee.groupby(['GENDER','RACE']).agg({'BASE_SALARY':['min', 'max'],
                                         'YEARS_EXPERIENCE':['mean','median']})

BASE_SALARY            \
                                                 min       max   
GENDER RACE                                                      
Female American Indian or Alaskan Native     26125.0   98536.0   
       Asian/Pacific Islander                26125.0  130416.0   
       Black or African American             24960.0  150416.0   
       Hispanic/Latino                       26125.0  126115.0   
       Others                                63785.0   63785.0   
       White                                 27955.0  178331.0   
Male   American Indian or Alaskan Native     26125.0   81239.0   
       Asian/Pacific Islander                27914.0  163228.0   
       Black or African American             26125.0  275000.0   
       Hispanic/Latino                       26104.0  165216.0   
       Others                                38771.0   38771.0   
       White                                 26125.0  210588.0   

                                         YEARS_EXPERIENCE         
                                                     mean median  
GENDER RACE                                                       
Female American Indian or Alaskan Native        12.583333  12.95  
       Asian/Pacific Islander                   14.470000  12.15  
       Black or African American                13.652131  11.00  
       Hispanic/Latino                          11.063309   8.70  
       Others                                    1.100000   1.10  
       White                                    13.018519  10.05  
Male   American Indian or Alaskan Native        13.560000  10.70  
       Asian/Pacific Islander                   13.519481  11.80  
       Black or African American                12.937722  10.40  
       Hispanic/Latino                          12.816716  10.50  
       Others                                    4.000000   4.00  
       White                                    17.869120  15.50

In [14]:
employee.groupby(['GENDER','RACE']).agg({'BASE_SALARY':['mean', 'median'], 
                                         'YEARS_EXPERIENCE':'count'})

BASE_SALARY           \
                                                  mean   median   
GENDER RACE                                                       
Female American Indian or Alaskan Native  60238.800000  58855.0   
       Asian/Pacific Islander             63226.300000  57227.5   
       Black or African American          48915.421233  44491.0   
       Hispanic/Latino                    46503.316176  43087.0   
       Others                             63785.000000  63785.0   
       White                              66793.352941  62264.5   
Male   American Indian or Alaskan Native  60305.400000  60347.0   
       Asian/Pacific Islander             61033.906667  55461.0   
       Black or African American          51082.074074  46486.5   
       Hispanic/Latino                    54782.819018  54090.5   
       Others                             38771.000000  38771.0   
       White                              63940.388119  62540.0   

                                         YEARS_EXPERIENCE  
                                                    count  
GENDER RACE                                                
Female American Indian or Alaskan Native                6  
       Asian/Pacific Islander                          30  
       Black or African American                      305  
       Hispanic/Latino                                139  
       Others                                           1  
       White                                          108  
Male   American Indian or Alaskan Native                5  
       Asian/Pacific Islander                          77  
       Black or African American                      395  
       Hispanic/Latino                                341  
       Others                                           1  
       White                                          557

## Not explicitly writing aggregate columns
It isn't necessary to put your aggregating columns in a list after calling group by. If you just call a method, pandas will attempt to aggregate all the rest of your columns.

In [15]:
# drops all columns that it can't find a mean.
employee.groupby('RACE').mean()

BASE_SALARY  YEARS_EXPERIENCE
RACE                                                             
American Indian or Alaskan Native  60272.100000         13.027273
Asian/Pacific Islander             61660.304762         13.785981
Black or African American          50137.801493         13.249000
Hispanic/Latino                    52345.562771         12.308958
Others                             51278.000000          2.550000
White                              64419.799012         17.081353

In [16]:
employee.groupby(['RACE', 'GENDER']).agg(['mean', 'min'])

BASE_SALARY           \
                                                  mean      min   
RACE                              GENDER                          
American Indian or Alaskan Native Female  60238.800000  26125.0   
                                  Male    60305.400000  26125.0   
Asian/Pacific Islander            Female  63226.300000  26125.0   
                                  Male    61033.906667  27914.0   
Black or African American         Female  48915.421233  24960.0   
                                  Male    51082.074074  26125.0   
Hispanic/Latino                   Female  46503.316176  26125.0   
                                  Male    54782.819018  26104.0   
Others                            Female  63785.000000  63785.0   
                                  Male    38771.000000  38771.0   
White                             Female  66793.352941  27955.0   
                                  Male    63940.388119  26125.0   

                                         YEARS_EXPERIENCE       
                                                     mean  min  
RACE                              GENDER                        
American Indian or Alaskan Native Female        12.583333  2.8  
                                  Male          13.560000  3.7  
Asian/Pacific Islander            Female        14.470000  1.0  
                                  Male          13.519481  0.5  
Black or African American         Female        13.652131  0.5  
                                  Male          12.937722  0.5  
Hispanic/Latino                   Female        11.063309  0.6  
                                  Male          12.816716  0.5  
Others                            Female         1.100000  1.1  
                                  Male           4.000000  4.0  
White                             Female        13.018519  0.8  
                                  Male          17.869120  0.6

# Your Turn
Use the City of Houston employee dataset for the first few problems

In [17]:
employee = pd.read_csv('data/employee.csv', index_col='UNIQUE_ID', parse_dates=['HIRE_DATE', 'JOB_DATE'])
employee['YEARS_EXPERIENCE'] = (pd.Timestamp('2016-12-1') - employee['HIRE_DATE']) / pd.Timedelta(1, 'Y')
employee['YEARS_EXPERIENCE'] = employee['YEARS_EXPERIENCE'].round(1)
employee['EXPERIENCE_LEVEL'] =  pd.cut(employee['YEARS_EXPERIENCE'], 
                                       bins=[0, 5, 15, 100], 
                                       labels=['Novice', 'Experienced', 'Senior'])

### Problem 1
<span  style="color:green; font-size:16px">For every level of experience and gender find the average salary.</span>

In [19]:
employee.groupby('EXPERIENCE_LEVEL')['BASE_SALARY'].mean()

EXPERIENCE_LEVEL
Novice         44987.484000
Experienced    55237.875946
Senior         63685.987586
Name: BASE_SALARY, dtype: float64

In [18]:
# your code here
employee.head()

POSITION_TITLE                     DEPARTMENT  \
UNIQUE_ID                                                               
5906       ASSISTANT DIRECTOR (EX LVL)    Municipal Courts Department   
364                  LIBRARY ASSISTANT                        Library   
1286                    POLICE OFFICER  Houston Police Department-HPD   
8789                 ENGINEER/OPERATOR  Houston Fire Department (HFD)   
8542                       ELECTRICIAN    General Services Department   

           BASE_SALARY             RACE EMPLOYMENT_TYPE  GENDER  \
UNIQUE_ID                                                         
5906          121862.0  Hispanic/Latino       Full Time  Female   
364            26125.0  Hispanic/Latino       Full Time  Female   
1286           45279.0            White       Full Time    Male   
8789           63166.0            White       Full Time    Male   
8542           56347.0            White       Full Time    Male   

          EMPLOYMENT_STATUS  HIRE_DATE   JOB_DATE  YEARS_EXPERIENCE  \
UNIQUE_ID                                                             
5906                 Active 2006-06-12 2012-10-13              10.5   
364                  Active 2000-07-19 2010-09-18              16.4   
1286                 Active 2015-02-03 2015-02-03               1.8   
8789                 Active 1982-02-08 1991-05-25              34.8   
8542                 Active 1989-06-19 1994-10-22              27.5   

          EXPERIENCE_LEVEL  
UNIQUE_ID                   
5906           Experienced  
364                 Senior  
1286                Novice  
8789                Senior  
8542                Senior

### Problem 2
<span  style="color:green; font-size:16px">Repeat problem 1 but return it as a three column DataFrame.</span>

In [20]:
# your code here
employee.groupby('EXPERIENCE_LEVEL',as_index=False)['BASE_SALARY'].mean()

EXPERIENCE_LEVEL   BASE_SALARY
0           Novice  44987.484000
1      Experienced  55237.875946
2           Senior  63685.987586

### Problem 3
<span  style="color:green; font-size:16px">For every race and level of experience find the min, max, mean, median of the salary and the size of the group.</span>

In [21]:
# your code here
employee.groupby(['RACE', 'EXPERIENCE_LEVEL']).agg(['mean', 'min','max','median'])

BASE_SALARY           \
                                                            mean      min   
RACE                              EXPERIENCE_LEVEL                          
American Indian or Alaskan Native Novice            62330.500000  26125.0   
                                  Experienced       47828.000000  26125.0   
                                  Senior            71687.000000  58855.0   
Asian/Pacific Islander            Novice            48441.250000  27914.0   
                                  Experienced       67407.317073  26125.0   
                                  Senior            63701.050000  31408.0   
Black or African American         Novice            42065.415301  24960.0   
                                  Experienced       48208.552174  26125.0   
                                  Senior            57612.408560  26125.0   
Hispanic/Latino                   Novice            42079.701493  26104.0   
                                  Experienced       53466.531792  26125.0   
                                  Senior            59969.419355  26125.0   
Others                            Novice            51278.000000  38771.0   
White                             Novice            53178.952000  27352.0   
                                  Experienced       62063.600939  26125.0   
                                  Senior            71508.929368  29286.0   

                                                                       \
                                                         max   median   
RACE                              EXPERIENCE_LEVEL                      
American Indian or Alaskan Native Novice             98536.0  62330.5   
                                  Experienced        60347.0  52420.0   
                                  Senior             81239.0  73327.0   
Asian/Pacific Islander            Novice             93491.0  46464.5   
                                  Experienced       130416.0  60925.0   
                                  Senior            163228.0  54833.0   
Black or African American         Novice            275000.0  35630.0   
                                  Experienced       110686.0  45063.5   
                                  Senior            180416.0  57815.0   
Hispanic/Latino                   Novice            165216.0  40365.0   
                                  Experienced       121862.0  54600.0   
                                  Senior            142006.0  63166.0   
Others                            Novice             63785.0  51278.0   
White                             Novice            150416.0  45791.0   
                                  Experienced       199596.0  57304.0   
                                  Senior            210588.0  66614.0   

                                                   YEARS_EXPERIENCE        \
                                                               mean   min   
RACE                              EXPERIENCE_LEVEL                          
American Indian or Alaskan Native Novice                   3.250000   2.8   
                                  Experienced             10.700000   8.3   
                                  Senior                  20.825000  17.2   
Asian/Pacific Islander            Novice                   2.657692   0.5   
                                  Experienced             10.621951   6.5   
                                  Senior                  24.262500  15.5   
Black or African American         Novice                   2.376768   0.5   
                                  Experienced              9.904741   5.1   
                                  Senior                  24.095556  15.1   
Hispanic/Latino                   Novice                   2.416197   0.5   
                                  Experienced              9.995954   5.2   
                                  Senior                  23.247879  15.1   
Others                            Novice      

### Problem 4
<span  style="color:green; font-size:16px">For each department and gender find the number of unique position titles, the total number of employees and the average salary.</span>

In [23]:
# your code here
employee.head()

POSITION_TITLE                     DEPARTMENT  \
UNIQUE_ID                                                               
5906       ASSISTANT DIRECTOR (EX LVL)    Municipal Courts Department   
364                  LIBRARY ASSISTANT                        Library   
1286                    POLICE OFFICER  Houston Police Department-HPD   
8789                 ENGINEER/OPERATOR  Houston Fire Department (HFD)   
8542                       ELECTRICIAN    General Services Department   

           BASE_SALARY             RACE EMPLOYMENT_TYPE  GENDER  \
UNIQUE_ID                                                         
5906          121862.0  Hispanic/Latino       Full Time  Female   
364            26125.0  Hispanic/Latino       Full Time  Female   
1286           45279.0            White       Full Time    Male   
8789           63166.0            White       Full Time    Male   
8542           56347.0            White       Full Time    Male   

          EMPLOYMENT_STATUS  HIRE_DATE   JOB_DATE  YEARS_EXPERIENCE  \
UNIQUE_ID                                                             
5906                 Active 2006-06-12 2012-10-13              10.5   
364                  Active 2000-07-19 2010-09-18              16.4   
1286                 Active 2015-02-03 2015-02-03               1.8   
8789                 Active 1982-02-08 1991-05-25              34.8   
8542                 Active 1989-06-19 1994-10-22              27.5   

          EXPERIENCE_LEVEL  
UNIQUE_ID                   
5906           Experienced  
364                 Senior  
1286                Novice  
8789                Senior  
8542                Senior

In [39]:
employee.groupby(['DEPARTMENT','GENDER']).agg({ 'POSITION_TITLE':['nunique','count'],'BASE_SALARY':'mean'})

POSITION_TITLE          BASE_SALARY
                                             nunique count           mean
DEPARTMENT                     GENDER                                    
Admn. & Regulatory Affairs     Female             16    22   48758.181818
                               Male                7     7   57592.285714
City Controller's Office       Female              2     4   58979.500000
                               Male                1     1   42640.000000
City Council                   Female              5     7   59260.000000
                               Male                4     4   58491.500000
Convention and Entertainment   Female              1     1   38397.000000
Dept of Neighborhoods (DON)    Female              8     8   50577.500000
                               Male                6     9   43995.444444
Finance                        Female              4     4   83254.250000
                               Male                4     6   77247.666667
Fleet Management Department    Female              1     3   52992.000000
                               Male               11    33   43176.333333
General Services Department    Female              2     2   60043.000000
                               Male               11    20   50421.100000
Health & Human Services        Female             47    83   48853.392405
                               Male               23    27   58481.888889
Housing and Community Devp.    Female              9    16   64415.812500
                               Male                4     6   49275.250000
Houston Airport System (HAS)   Female             24    36   53174.194444
                               Male               37    70   54358.171429
Houston Emergency Center (HEC) Female              9    17   47345.882353
                               Male                4     6   43780.333333
Houston Fire Department (HFD)  Female             13    21   52853.047619
                               Male               26   363   59930.564470
Houston Information Tech Svcs  Female              1     1   74951.000000
                               Male                6     8   76258.000000
Houston Police Department-HPD  Female             38   155   52219.925170
                               Male               27   483   63032.841121
Human Resources Dept.          Female             11    19   55990.631579
                               Male                5     5   57532.400000
Legal Department               Female              4     9   91479.888889
                               Male                6     8  120124.125000
Library                        Female             16    28   43627.090909
                               Male                7     8   45016.125000
Mayor's Office                 Female              3     3   88902.666667
                               Male                2     2   82870.000000
Municipal Courts Department    Female             10    21   53557.550000
                               Male                6     7   54371.500000
Parks & Recreation             Female             15    23   40361.055556
                               Male               20    51   38396.243243
Planning & Development         Female              2     3   58234.333333
                               Male                2     4   52363.500000
Public Works & Engineering-PWE Female             55   108   51023.301887
                               Male               70   235   49839.965957
Solid Waste Management         Female              7     9   39748.222222
                               Male               11    34   42079.970588

### Problem 5
<span  style="color:green; font-size:16px">For each department, race and gender find the maximum years of experience and salary.</span>

In [29]:
employee.head()

POSITION_TITLE                     DEPARTMENT  \
UNIQUE_ID                                                               
5906       ASSISTANT DIRECTOR (EX LVL)    Municipal Courts Department   
364                  LIBRARY ASSISTANT                        Library   
1286                    POLICE OFFICER  Houston Police Department-HPD   
8789                 ENGINEER/OPERATOR  Houston Fire Department (HFD)   
8542                       ELECTRICIAN    General Services Department   

           BASE_SALARY             RACE EMPLOYMENT_TYPE  GENDER  \
UNIQUE_ID                                                         
5906          121862.0  Hispanic/Latino       Full Time  Female   
364            26125.0  Hispanic/Latino       Full Time  Female   
1286           45279.0            White       Full Time    Male   
8789           63166.0            White       Full Time    Male   
8542           56347.0            White       Full Time    Male   

          EMPLOYMENT_STATUS  HIRE_DATE   JOB_DATE  YEARS_EXPERIENCE  \
UNIQUE_ID                                                             
5906                 Active 2006-06-12 2012-10-13              10.5   
364                  Active 2000-07-19 2010-09-18              16.4   
1286                 Active 2015-02-03 2015-02-03               1.8   
8789                 Active 1982-02-08 1991-05-25              34.8   
8542                 Active 1989-06-19 1994-10-22              27.5   

          EXPERIENCE_LEVEL  
UNIQUE_ID                   
5906           Experienced  
364                 Senior  
1286                Novice  
8789                Senior  
8542                Senior

In [30]:
# your code here
employee.groupby(['DEPARTMENT','RACE','GENDER']).agg({'BASE_SALARY':'max','YEARS_EXPERIENCE':'max'})

BASE_SALARY  \
DEPARTMENT                     RACE                              GENDER                
Admn. & Regulatory Affairs     Asian/Pacific Islander            Female     130416.0   
                               Black or African American         Female      72741.0   
                                                                 Male        30098.0   
                               Hispanic/Latino                   Female      47341.0   
                                                                 Male        35318.0   
                               White                             Female      62129.0   
                                                                 Male       140416.0   
City Controller's Office       Asian/Pacific Islander            Female      59077.0   
                               Black or African American         Female      57054.0   
                                                                 Male        42640.0   
                               Hispanic/Latino                   Female      64251.0   
City Council                   Black or African American         Female     100000.0   
                                                                 Male        54000.0   
                               Hispanic/Latino                   Female      35837.0   
                                                                 Male            NaN   
                               White                             Female      77000.0   
                                                                 Male        62983.0   
Convention and Entertainment   Hispanic/Latino                   Female      38397.0   
Dept of Neighborhoods (DON)    American Indian or Alaskan Native Male        26125.0   
                               Black or African American         Female      64424.0   
                                                                 Male        89221.0   
                               Hispanic/Latino                   Female      54829.0   
                                                                 Male        43742.0   
                               White                             Female      65832.0   
Finance                        Black or African American         Female      81458.0   
                                                                 Male        96272.0   
                               Hispanic/Latino                   Female      95773.0   
                                                                 Male        93168.0   
Fleet Management Department    Asian/Pacific Islander            Male        46010.0   
                               Black or African American         Female      56940.0   
...                                                                              ...   
Municipal Courts Department    Hispanic/Latino                   Female     121862.0   
                                                                 Male        31304.0   
                               White                             Female     117176.0   
                                                                 Male        88736.0   
Parks & Recreation             Asian/Pacific Islander            Male        48651.0   
                               Black or African American         Female      57533.0   
                                                                 Male        56014.0   
                               Hispanic/Latino                   Female      83916.0   
                                                                 Male        72280.0   
                               White                             Female          NaN   
                                                                 Male        85055.0   
Planning & Development         Asian/Pacific Islander            Female      65402.0   
                               Black or African American         Female      68762.0   
                         

# Solutions

### Problem 1
<span  style="color:green; font-size:16px">For every level of experience and gender find the average salary.</span>

In [31]:
employee.groupby(['GENDER', 'EXPERIENCE_LEVEL'])['BASE_SALARY'].mean()

GENDER  EXPERIENCE_LEVEL
Female  Novice              45746.932099
        Experienced         53010.630542
        Senior              56268.731132
Male    Novice              44623.488166
        Experienced         56225.061135
        Senior              66751.208577
Name: BASE_SALARY, dtype: float64

### Problem 2
<span  style="color:green; font-size:16px">Repeat problem 1 but return it as a three column DataFrame.</span>

In [32]:
employee.groupby(['GENDER', 'EXPERIENCE_LEVEL'])['BASE_SALARY'].mean().reset_index()

GENDER EXPERIENCE_LEVEL   BASE_SALARY
0  Female           Novice  45746.932099
1  Female      Experienced  53010.630542
2  Female           Senior  56268.731132
3    Male           Novice  44623.488166
4    Male      Experienced  56225.061135
5    Male           Senior  66751.208577

In [33]:
employee.groupby(['GENDER', 'EXPERIENCE_LEVEL'], as_index=False)['BASE_SALARY'].mean()

GENDER EXPERIENCE_LEVEL   BASE_SALARY
0  Female           Novice  45746.932099
1  Female      Experienced  53010.630542
2  Female           Senior  56268.731132
3    Male           Novice  44623.488166
4    Male      Experienced  56225.061135
5    Male           Senior  66751.208577

### Problem 3
<span  style="color:green; font-size:16px">For every race and level of experience find the min, max, mean, median of the salary and the size of the group.</span>

In [34]:
employee.groupby(['RACE', 'EXPERIENCE_LEVEL'])['BASE_SALARY'].agg(['min', 'max','mean','median','size']).astype('int')

min     max   mean  \
RACE                              EXPERIENCE_LEVEL                         
American Indian or Alaskan Native Novice            26125   98536  62330   
                                  Experienced       26125   60347  47828   
                                  Senior            58855   81239  71687   
Asian/Pacific Islander            Novice            27914   93491  48441   
                                  Experienced       26125  130416  67407   
                                  Senior            31408  163228  63701   
Black or African American         Novice            24960  275000  42065   
                                  Experienced       26125  110686  48208   
                                  Senior            26125  180416  57612   
Hispanic/Latino                   Novice            26104  165216  42079   
                                  Experienced       26125  121862  53466   
                                  Senior            26125  142006  59969   
Others                            Novice            38771   63785  51278   
White                             Novice            27352  150416  53178   
                                  Experienced       26125  199596  62063   
                                  Senior            29286  210588  71508   

                                                    median  size  
RACE                              EXPERIENCE_LEVEL                
American Indian or Alaskan Native Novice             62330     2  
                                  Experienced        52420     5  
                                  Senior             73327     4  
Asian/Pacific Islander            Novice             46464    26  
                                  Experienced        60925    41  
                                  Senior             54833    40  
Black or African American         Novice             35630   198  
                                  Experienced        45063   232  
                                  Senior             57815   270  
Hispanic/Latino                   Novice             40365   142  
                                  Experienced        54600   173  
                                  Senior             63166   165  
Others                            Novice             51278     2  
White                             Novice             45791   126  
                                  Experienced        57304   214  
                                  Senior             66614   325

### Problem 4
<span  style="color:green; font-size:16px">For each department and gender find the number of unique position titles, the total number of employees and the average salary.</span>

In [35]:
employee.groupby(['DEPARTMENT', 'GENDER']).agg({'POSITION_TITLE':['nunique','size'],
                                                'BASE_SALARY':'mean'}).astype(int).head(15)

POSITION_TITLE      BASE_SALARY
                                           nunique size        mean
DEPARTMENT                   GENDER                                
Admn. & Regulatory Affairs   Female             16   22       48758
                             Male                7    7       57592
City Controller's Office     Female              2    4       58979
                             Male                1    1       42640
City Council                 Female              5    7       59260
                             Male                4    4       58491
Convention and Entertainment Female              1    1       38397
Dept of Neighborhoods (DON)  Female              8    8       50577
                             Male                6    9       43995
Finance                      Female              4    4       83254
                             Male                4    6       77247
Fleet Management Department  Female              1    3       52992
                             Male               11   33       43176
General Services Department  Female              2    2       60043
                             Male               11   20       50421

### Problem 5
<span  style="color:green; font-size:16px">For each department, race and gender find the maximum years of experience and salary.</span>

In [36]:
employee.groupby(['DEPARTMENT','RACE','GENDER'])['BASE_SALARY', 'YEARS_EXPERIENCE'].max().head(20)

BASE_SALARY  \
DEPARTMENT                   RACE                              GENDER                
Admn. & Regulatory Affairs   Asian/Pacific Islander            Female     130416.0   
                             Black or African American         Female      72741.0   
                                                               Male        30098.0   
                             Hispanic/Latino                   Female      47341.0   
                                                               Male        35318.0   
                             White                             Female      62129.0   
                                                               Male       140416.0   
City Controller's Office     Asian/Pacific Islander            Female      59077.0   
                             Black or African American         Female      57054.0   
                                                               Male        42640.0   
                             Hispanic/Latino                   Female      64251.0   
City Council                 Black or African American         Female     100000.0   
                                                               Male        54000.0   
                             Hispanic/Latino                   Female      35837.0   
                                                               Male            NaN   
                             White                             Female      77000.0   
                                                               Male        62983.0   
Convention and Entertainment Hispanic/Latino                   Female      38397.0   
Dept of Neighborhoods (DON)  American Indian or Alaskan Native Male        26125.0   
                             Black or African American         Female      64424.0   

                                                                       YEARS_EXPERIENCE  
DEPARTMENT                   RACE                              GENDER                    
Admn. & Regulatory Affairs   Asian/Pacific Islander            Female              15.5  
                             Black or African American         Female              23.3  
                                                               Male                 4.7  
                             Hispanic/Latino                   Female              12.1  
                                                               Male                 9.0  
                             White                             Female              21.4  
                                                               Male                15.5  
City Controller's Office     Asian/Pacific Islander            Female               3.5  
                             Black or African American         Female              24.5  
                                                               Male                 3.3  
                             Hispanic/Latino                   Female              26.0  
City Council                 Black or African American         Female              11.7  
                                                               Male                 1.0  
                             Hispanic/Latino                   Female               3.2  
                                                               Male                 1.5  
                             White                             Female               4.9  
                                                               Male                 3.9  
Convention and Entertainment Hispanic/Latino                   Female              15.6  
Dept of Neighborhoods (DON)  American Indian or Alaskan Native Male                 3.7  
                             Black or African American         Female              37.2